In [1]:
import pandas as pd
import numpy as np
import re, json, os  
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  


In [4]:
ctx=context() 
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 

In [ ]:
for i in range(1,9):
    print(f"""
		IF @SUM_4_1_{i} > @ANS_4_1
		BEGIN
			INSERT INTO @ErrorTable  
			SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
			, ('The sum provided for ' + identifier_text + ' should not exceed the value of 4.1 : '  + QuestionText) Error, identifier_text
			FROM vw_OrgSubQuestions WHERE  identifier_text='4.1.{i}' AND PK_OrgSubmission = @PK_OrgSubmission  
		END""")
 

In [41]:
with open(ctx.get_tempalte_dir()+'sum.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}],SectionNum, SortPos FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO' AND IdText LIKE '4.1._._' ;
""", connstr) 

df['{var}'] = df['{var}'].apply(lambda s : '@SUM_'+re.sub('_[a-z]$','',s))
df['{idt}'] = df['{idt}'].apply(lambda s : f'\'{s}\'') 
df = df.groupby('{var}', as_index=False).agg({'{idt}':list})
df['sql']=[s.replace('{idt}',','.join(x)).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df 

,{var},{idt},sql
0,@SUM_4_1_1,"['4.1.1.a', '4.1.1.b', '4.1.1.c']",DECLARE @SUM_4_1_1 INT = (SELECT CASE @PK_Que...
1,@SUM_4_1_2,"['4.1.2.a', '4.1.2.b', '4.1.2.c']",DECLARE @SUM_4_1_2 INT = (SELECT CASE @PK_Que...
2,@SUM_4_1_3,"['4.1.3.a', '4.1.3.b', '4.1.3.c']",DECLARE @SUM_4_1_3 INT = (SELECT CASE @PK_Que...
3,@SUM_4_1_4,"['4.1.4.a', '4.1.4.b', '4.1.4.c']",DECLARE @SUM_4_1_4 INT = (SELECT CASE @PK_Que...
4,@SUM_4_1_5,"['4.1.5.a', '4.1.5.b', '4.1.5.c']",DECLARE @SUM_4_1_5 INT = (SELECT CASE @PK_Que...
5,@SUM_4_1_6,"['4.1.6.a', '4.1.6.b', '4.1.6.c']",DECLARE @SUM_4_1_6 INT = (SELECT CASE @PK_Que...
6,@SUM_4_1_7,"['4.1.7.a', '4.1.7.b', '4.1.7.c']",DECLARE @SUM_4_1_7 INT = (SELECT CASE @PK_Que...
7,@SUM_4_1_8,"['4.1.8.a', '4.1.8.b', '4.1.8.c']",DECLARE @SUM_4_1_8 INT = (SELECT CASE @PK_Que...


In [8]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}],SectionNum, SortPos FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO' AND Q_TypeId IN (2,17)  ORDER BY  SectionNum, SortPos;
""", connstr) 

df['sql']=s
df['sql']=[x.replace('{idt}',y).replace('{var}',z) for x,y,z in zip(df['sql'], df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df

,{idt},{var},SectionNum,SortPos,sql
0,1.2.1,1_2_1,2,2,DECLARE @ANS_1_2_1 INT = (SELECT TOP 1 CASE @...
1,1.2.2,1_2_2,2,3,DECLARE @ANS_1_2_2 INT = (SELECT TOP 1 CASE @...
2,1.2.3,1_2_3,2,4,DECLARE @ANS_1_2_3 INT = (SELECT TOP 1 CASE @...
3,1.2.4,1_2_4,2,5,DECLARE @ANS_1_2_4 INT = (SELECT TOP 1 CASE @...
4,1.2.5,1_2_5,2,6,DECLARE @ANS_1_2_5 INT = (SELECT TOP 1 CASE @...
...,...,...,...,...,...
168,10.5.2,10_5_2,13,14,DECLARE @ANS_10_5_2 INT = (SELECT TOP 1 CASE ...
169,10.5.3,10_5_3,13,15,DECLARE @ANS_10_5_3 INT = (SELECT TOP 1 CASE ...
170,10.6.1,10_6_1,13,17,DECLARE @ANS_10_6_1 INT = (SELECT TOP 1 CASE ...
171,10.6.2,10_6_2,13,18,DECLARE @ANS_10_6_2 INT = (SELECT TOP 1 CASE ...


In [36]:
options = Options()
options.add_argument("--start-minimized")  
options.add_argument("--window-size=820,680")  
driver = webdriver.Chrome(executable_path=ChromeDriverManager(log_level=0).install() ,options=options)  



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST driver version for 107.0.5304
Driver [C:\Users\timko\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


In [34]:
driver.get(f"https://dayman.cyber-balance.com/jira/login.jsp")  
driver.find_element(By.XPATH, '//input[contains(@id, "user")]').send_keys(ctx.config['user'])
driver.find_element(By.XPATH, '//input[contains(@id, "pass")]').send_keys(ctx.config['pass']) 
driver.find_element(By.XPATH, '//input[contains(@id, "submit")]').click() 

In [35]:
def parse_metrics(tnum, pk, met_parse='(\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?)\s+[A-Z]'):
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")  
    ele=driver.find_element_by_css_selector('td#descriptionArea')
    soup = BeautifulSoup(ele.get_attribute('innerHTML')) 
    le = soup.select('td.confluenceTd, p') 
    lod=[]
    for i,e in enumerate(le):
        m=re.search(met_parse,str(e.text))   
        if(m):  
            print(m.groups(1))
            lod.append({'identifier_text':m.groups(1)[0]
                ,'QuestionText':e.text.replace(m.groups(1)[0],'') 
                ,'FK_QuestionType':get_question_type(e.text, ctx)['PK_QuestionTypeId'] 
            })  
    df=pd.DataFrame(lod) 
    df['sortpos']=range(1,len(df)+1) 
    df['{PK_Question}']=range(pk,len(df)+pk) 
    return df 
df = parse_metrics('9308', 36421 )
df
#sql =df_to_sql(df, ctx)
#print(sql)

('3.1',)
('3.1.1',)
('3.1.2',)


,identifier_text,QuestionText,FK_QuestionType,sortpos,{PK_Question}
0,3.1,"Using the model defined in OMB M-21-31, provi...",2,1,36421
1,3.1.1,Of the assessment provided at the enterprise ...,2,2,36422
2,3.1.2,Please provide the number of HVAs that have a...,2,3,36423


In [68]:
def generate_code_from_db(qgroup):
    df=sql_todf("""
    SELECT 
idText as identifier_text 
,CASE WHEN idText='' THEN REPLACE(ID,'-','_')  ELSE REPLACE(idText,'.','_') END AS [{idt}]
,ID as [id]
,Q_TypeCode CTRLCODE
,FK_QuestionType
,PK_Question as [{PK_Question}]
,sortpos
,QTEXT as [{QuestionText}] 
    """+ f"FROM vwQuestions WHERE PK_QuestionGroup={qgroup}", connstr) 
    df['{idt}']=df['{idt}'].apply(lambda s: re.sub('_$','',s))
    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    return df, code


df,code=generate_code_from_db(4018) 
generate_section(4018, code)
 
#df[:10]


{4011: 'FismaForms/2023/2023_Q1_CIO_1A.aspx', 4012: 'FismaForms/2023/2023_Q1_CIO_1B.aspx', 4013: 'FismaForms/2023/2023_Q1_CIO_1C.aspx', 4014: 'FismaForms/2023/2023_Q1_CIO_1D.aspx', 4015: 'FismaForms/2023/2023_Q1_CIO_2.aspx', 4016: 'FismaForms/2023/2023_Q1_CIO_3A.aspx', 4017: 'FismaForms/2023/2023_Q1_CIO_4.aspx', 4018: 'FismaForms/2023/2023_Q1_CIO_5.aspx', 4019: 'FismaForms/2023/2023_Q1_CIO_6.aspx', 4020: 'FismaForms/2023/2023_Q1_CIO_7a.aspx', 4021: 'FismaForms/2023/2023_Q1_CIO_8.aspx', 4022: 'FismaForms/2023/2023_Q1_CIO_9.aspx', 4023: 'FismaForms/2023/2023_Q1_CIO_10.aspx'}


In [64]:
def generate_section(section, code=''): 
    df=sql_todf("SELECT DISTINCT PK_QuestionGroup, ASPX FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO'", connstr).set_index('PK_QuestionGroup' )
    d=df.T.to_dict(orient='records')[0] 
    print(d)
    path=f'C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\{d[section]}'
    with open(path, 'r', encoding='utf-16') as f:
        s=f.read() 
    pname = d[section][d[section].rfind('/')+1:]
    if '<!--questions-->' in s:
        q=re.findall('<!--questions-->.+<!--\/questions-->',s,re.DOTALL)[0]
        s=s.replace(q,f'<!--questions-->\n{code}\n<!--/questions-->')
        with open(f"{ctx.get_dest()}{pname}", 'w', encoding='utf-16') as f:
            f.write(s) 



{4011: 'FismaForms/2023/2023_Q1_CIO_1A.aspx', 4012: 'FismaForms/2023/2023_Q1_CIO_1B.aspx', 4013: 'FismaForms/2023/2023_Q1_CIO_1C.aspx', 4014: 'FismaForms/2023/2023_Q1_CIO_1D.aspx', 4015: 'FismaForms/2023/2023_Q1_CIO_2.aspx', 4016: 'FismaForms/2023/2023_Q1_CIO_3A.aspx', 4017: 'FismaForms/2023/2023_Q1_CIO_4.aspx', 4018: 'FismaForms/2023/2023_Q1_CIO_5.aspx', 4019: 'FismaForms/2023/2023_Q1_CIO_6.aspx', 4020: 'FismaForms/2023/2023_Q1_CIO_7a.aspx', 4021: 'FismaForms/2023/2023_Q1_CIO_8.aspx', 4022: 'FismaForms/2023/2023_Q1_CIO_9.aspx', 4023: 'FismaForms/2023/2023_Q1_CIO_10.aspx'}


In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
df=pd.DataFrame(range(1,2))
df["identifier_text"]=df[0].apply(lambda s : '3.3.'+str(s))
df['CTRLCODE']='LABEL'
df['m']=[['a Tier EL0 Not Effective','b Tier EL1 Basic','c Tier EL2 Intermediate','d Tier EL3 Advanced'] for i in range(len(df))]
lbls=df
df=df.explode('m')
df=pd.concat([lbls,df]) 
df['CTRLCODE']=df['m'].apply(lambda s: 2 if type(s)==str else 18)
df.sort_values([0,'CTRLCODE'], ascending=[True,False], inplace=True) 
df['m']=df['m'].apply(lambda s: s if type(s)!=list else '') 
df['QuestionText']=  df['m'].apply(lambda s: s[2:])
df["identifier_text"]=df["identifier_text"]+ ''+ df['m'].apply(lambda s: s.split(' ')[0])
df['{PK_Question}'] = range(37250,len(df)+37250)
df['sortpos'] = range(50,len(df)+50) 
df['FK_QuestionType'] = df['CTRLCODE']
df['CTRLCODE'] = df['FK_QuestionType'].apply(lambda s: 'CNT' if s==2 else 'LABEL')  
df['{idt}'] = df['identifier_text'].str.replace('.','_')
df.drop(columns=[0,'m'], inplace=True)
df
 


In [ ]:
p=questionnaire_parser(ctx)
df=p.parse()
df.to_html('parsed.html')
df.to_csv('parsed.csv')

In [ ]:
input="""
What is the approximate time in years since deployment of this HVA?
 
""".split('\n')

pr = picklist_recommender(connstr)
r=pr.recommend(input, threshhold=.8, usecosine_sim=False)
r